<a href="https://colab.research.google.com/github/tktmyd/ipynbs/blob/main/fdm/FDM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1次元問題の差分法シミュレーション：実践編

In [ ]:
import sys
if 'google.colab' in sys.modules:
    print("Installing pygmt on Google Colab. It may take a few minutes.")
    !pip install -q condacolab > /dev/null
    import condacolab
    condacolab.install()

In [ ]:
%%bash
sed -i '/^python_abi/d;/^python[[:space:]]/d' /usr/local/conda-meta/pinned || true
mamba install -y -n base python=3.12 gmt pygmt >& /dev/null
# 強制的にPyGMTを認識させる
ln -sf /usr/local/lib/python3.12/site-packages/pygmt \
       /usr/local/lib/python3.12/dist-packages/
ln -sf /usr/local/lib/python3.12/site-packages/pygmt-*.dist-info \
       /usr/local/lib/python3.12/dist-packages/ 2>/dev/null || true

In [ ]:
import numpy as np
import pygmt

これまでに導いた1次元差分法のアルゴリズム

$$
\begin{align}
	 &	\sigma_i \leftarrow \sigma_i
	 +
	 G_i \frac{v_i - v_{i-1} }{\Delta x} \Delta t
	\\
	 & v_i  \leftarrow  v_i
	 +
	 \frac{1}{\rho_i }
	 \frac{\sigma_{i+1} - \sigma_{i} }{\Delta x} \Delta t
\end{align}
$$
を使って，実際に波動の伝播をシミュレーションしてみましょう．

これから作るプログラムは，

- パラメタの設定と変数（メモリ）の確保
- 初期条件の設定
- 差分法による時間発展の計算とデータ出力

という部分からなります．導出したアルゴリズムはこのうちの一番最後の部分ですが，その計算のまえにいくらかの準備が必要です．

この数値シミュレーションでは，空間 $x$ と時間 $t$ を離散化していますから，それぞれの離散化幅 $\Delta x$, $\Delta t$ を定める必要があります．また，無限に広い領域の計算をするわけにはいきませんから，空間と時間それぞれについてのグリッドの個数も定めます．

これらのパラメタをどうやって適切に定めるのか，は重要な話題なのですが，ここではまず実践ということで，適当に値を設定してみます．

In [ ]:
dx = 0.2               # 空間グリッドサイズ (km)
dt = 0.05              # 時間ステップサイズ (s)
nx = 1001              # 空間グリッド数
nt = 401               # 時間ステップ数

V   = np.zeros(nx+1)   # 変位速度 V[0] ... V[nx] をゼロで確保．以下同様
S   = np.zeros(nx+2)   # 応力
rho = np.zeros(nx+1)   # 質量密度
G   = np.zeros(nx+1)   # 弾性係数

ここでは，`np.zeros` 関数を用いてメモリを確保しました．この関数は引数に与えられた個数リストを確保し，その値をすべて `0` で初期化します．プログラミング言語ではギリシャ文字は使いにくい（Pythonでは使えないこともないが，入力が不便）のため，$\sigma$ は `S`, $\rho$ は `rho` としました．

続けて，媒質と初期条件の設定です．数値シミュレーションの中では，質量密度と剛性率が必要です．一方，地震学的には剛性率そのものよりも地震波速度のほうがわかりやすいので，ここでは
$\beta = \sqrt{G/\rho}$ として，地震波速度 $\beta$ を通じて媒質を特徴づけることにします．

In [ ]:
# 媒質の設定
for i in range(nx+1):
    beta   = 4.0                   # 仮定する地震波速度 (km/s)
    rho[i] = 2.7                   # 仮定する質量密度 (g/cm^3)
    G[i]   = rho[i] * beta * beta  # 剛性率に換算

x = np.linspace(0, dx*(nx-1), nx+1)

ここでは，地震波速度を 4 km/s の一定値，質量密度を 2.7 g/cm$^3$ の一定値としました．また，あとで用いるため，$x$軸にそった各セルの位置の座標値を，`x` というリストに設定しました．`np.linspace()` は指定範囲内で等間隔の数列を生成する関数です．

さらにつづけて初期条件を設定します．本当の断層運動による地震波の生成は，複数の等価体積力の組み合わせで得られるのですが，ここでは簡単のため，粒子速度$v$があらかじめ有限の値を持っていた，という状態を初期値として設定してみます．

In [ ]:
L  = 8.0               # 初期条件の波長 (km)

# 初期条件
hw = int(L/2/dx + 0.5) # 半値幅
for i in range(-hw, hw+1):
    V[int((nx - 1)/2)+1+i] = np.cos(np.pi/2.0*i/hw)**2

n = 0 # 時間ステップの値（ファイル名に利用）

# 結果出力ファイル
with open(f'out_{n*dt:06.2f}_.dat', 'w') as fp:
    for i in range(nx+1):
        print(f"{(i-1)*dx:12.4e} {V[i]:12.4e}", file=fp)

上記のコードは，変数 `L` で指定した長さの幅をもつ，$\cos$関数で表されるような滑らかなベル型の初期条件を $x$ 軸中央に設定しています．このコードを実行すると，$x$と$v$の値がファイル `out_000.00_.dat` に出力されますから，それを読み込んでプロットしてみましょう．

In [ ]:
d = np.loadtxt(f'out_000.00_.dat').T
print(d)

`np.loadtxt`は読み込んだファイルを1行ずつのリストにしますが，いま欲しいのは1列ずつのリストです．そこで，`loadtxt`のあとに `.T` （行列の転置）をつけることによって列ごとのデータに変換しています．ここまでくれば，プロットは簡単です．

In [ ]:
fig = pygmt.Figure()
with pygmt.config(MAP_GRID_PEN_PRIMARY = '0.25p,30/30/30,.'):
    fig.basemap(projection='X12c/6c', region=[0, 200, -0.1, 1.1],
            frame = ['WS', 'xafg+lx [km]',
                           'yafg+lvelocity amplitude'])
    fig.plot(x=d[0], y=d[1], pen='thicker,150/100/250')
fig.show()

たしかに，中央付近に初期条件が設定されていることがわかります．

それでは，いよいよスタガードグリッド差分法の中核部分です．

In [ ]:
# 時間発展
for n in range(1, nt+1):

    # 構成関係式
    for i in range(1, nx+1):
        dxV = ( V[i] - V[i-1] ) / dx # 変位速度の空間微分
        S[i] += G[i] * dxV * dt      # 応力の更新

    # 運動方程式
    for i in range(1, nx+1):
        dxS = ( S[i+1] - S[i] ) / dx # 応力の空間微分
        V[i] += dxS / rho[i] * dt    # 変位速度の更新

    # ( n % m == 0 ) は m 回に1回 True になることを利用して出力を間引き
    if n % 5 == 0:
        with open(f'out_{n*dt:06.2f}_.dat', 'w') as fp:
            for i in range(nx+1):
                print(f"{(i-1)*dx:12.4e} {V[i]:12.4e}", file=fp)

一番外側に時間についてのループがあり，`n=0, 1, ..., nt` まで繰り返します．内側には空間についてのループが2つあり，それぞれ構成関係式・運動方程式に対応しています．それぞれのループ内で行っている計算は，本ノートブックの先頭にかかれているアルゴリズムとまったく同じです．

一方，コードの後半では計算結果の出力をしています．毎時間ステップのデータを出力するとやや分量が多いため，`n % 5 == 0` という条件を課して，5ステップごとにデータを出力するようにしています．出力ファイルは，`out_時間_.dat` という名前で保存されます．このひとつひとつのデータは，初期条件をプロットしたスクリプトと同様に可視化することができます．ここでは，すべてのデータの可視化結果をつなげてアニメーションにしてみましょう．

1枚の画像を作る関数 `plot_wave`，複数画像からGIFアニメーションを作る関数 `gif_anim`，そしてそれらを用いてシミュレーション結果の可視化をまとめて行う `plot_anim` を作成します．

このうち，`gif_anim` の中では，外部ツール `ffmpeg` を使って画像からGIFアニメーションを作成しています．また，一時的に大量の画像ファイルを作成するため，`tempdir` モジュールによって一時ディレクトリを作成し，処理終了後には削除するようにしています．PyGMTによる可視化結果から`ffmpeg`を用いたアニメーション作成については，たとえば[こちら](https://tktmyd.github.io/pygmt-howto-jp/animation.html)に詳しい解説があります．

In [ ]:
def plot_wave(filename):
    """
    シミュレーション出力ファイル filename をプロットし，PyGMTのFigureオブジェクトを返す．
    """

    tim = filename.split('_')[-2]
    d = np.loadtxt(filename).T

    fig = pygmt.Figure()
    with pygmt.config(MAP_GRID_PEN_PRIMARY = '0.25p,30/30/30,.'):
        fig.basemap(projection='X12c/6c', region=[0, d[0][-1], -1.1, 1.1],
                frame = ['WS', 'xafg+lx [km]',
                               'yafg+lvelocity amplitude'])
        fig.plot(x=d[0], y=d[1], pen='thicker,150/100/250')
        fig.text(x=d[0][-1]*0.99, y=0.99, justify='RT', text=f't = {tim} s',
                 font='10p,Helvetica,Black')

    return fig

In [ ]:
def gif_movie(figs, dpi=200, crop='0.5c'):

    """
    PyGMTのFigureオブジェクトのリストからGifアニメーションを作成する．Jupyter Notebook上で表示されるオブジェクトを返す．

    Parameters
    ----------
    figs : list of Figure
        PyGMTのFigureオブジェクトのリスト
    dpi : int, optional
        解像度 (default: 300)
    crop : str, optional
        余白のトリミング量 (default: '0.5c')

    Returns
    -------
    HTML : IPython.display.HTML
        Gifアニメーション
    """
    from IPython import display as dd
    import tempfile
    import base64
    import os

    with tempfile.TemporaryDirectory() as tmpdir:
        for i, fig in enumerate(figs):
            figname = f'plot_{i:05d}.png'
            print(f'\rsaving figs ... ({(i+1)/len(figs)*100:5.1f}%)', end='')
            fig.savefig(os.path.join(tmpdir, figname), dpi=dpi, crop=crop)

        print(' Done.')

        cmd1 = f'ffmpeg -i {tmpdir}/plot_%5d.png '
        cmd2 = f' -vf "scale=800:-1,split [a][b];[a] palettegen [p];[b][p] paletteuse" '
        cmd3 = f' {tmpdir}/out.gif > /dev/null 2>&1'
        print(f'making gif ... ', end='')
        os.system(cmd1 + cmd2 + cmd3)
        print(' Done.')

        with open(f'{tmpdir}/out.gif', 'rb') as f:
            b64 = base64.b64encode(f.read()).decode("ascii")

    return dd.HTML(f'<img src="data:image/gif;base64,{b64}" width="80%"/>')

In [ ]:
def plot_anim(indir):
    """
    ディレクトリ indir にあるシミュレーション結果を全て可視化し，gifアニメーションを作成する．"""

    import glob
    figs = []
    for f in sorted(glob.glob(f'{indir}/out_*.dat')):
        figs.append(plot_wave(f))

    return gif_movie(figs)

完成したコードで，現在のディレクトリ（`.`）に出力済みの結果を可視化してみましょう．

In [ ]:
plot_anim('.')

1次元の波動は，（現実世界の地震波の特徴としてよく知られている）幾何減衰は存在せず，同じ形が地震波速度で平行移動していきます．この場合は中央に初期条件として設定した振幅1の波が，$x$軸正負方向に振幅が半分ずつの波として分裂し，一定速度で伝播していく様子が確認できます．

ここで一旦，数値シミュレーションで出力した結果ファイルを削除しておきましょう．

In [ ]:
! rm -f out_???.??_.dat

## 数値実験

前節までで一通りのプログラムを作成し，数値シミュレーションを実施できました．ここでは，波動伝播と数値シミュレーションについての理解を深めるため，て様々なパラメタを変えて動作させてみることにします．そこで，前節ではステップごとに複数のセルに分割していたコードを一つの関数に縮約し，かつ関数の引数としてある程度のパラメタ変更ができるようにします．

In [ ]:
def fdm1d(beta=np.ones(1002)*4.0, dx=0.2, dt=0.05, nt=401, L=8.0, skip=5):

    import tempfile

    nx = len(beta) - 1
    V   = np.zeros(nx+1)   # 変位速度 V[0] ... V[nx] をゼロで確保．以下同様
    S   = np.zeros(nx+2)   # 応力
    rho = np.zeros(nx+1)   # 質量密度
    G   = np.zeros(nx+1)   # 弾性係数

    # 媒質の設定
    for i in range(nx+1):
        rho[i] = 2.7                         # 仮定する質量密度 (g/cm^3)
        G[i]   = rho[i] * beta[i] * beta[i]  # 剛性率に換算

    x = np.linspace(0, dx*(nx-1), nx+1)

    # 初期条件
    hw = int(L/2/dx + 0.5) # 半値幅
    for i in range(-hw, hw+1):
        V[int((nx - 1)/2)+1+i] = np.cos(np.pi/2.0*i/hw)**2

    n = 0 # 時間ステップの値（ファイル名に利用）

    # 出力用ディレクトリ作成
    with tempfile.TemporaryDirectory() as tmpd:

        # 結果出力ファイル
        with open(f'{tmpd}/out_{n*dt:06.2f}_.dat', 'w') as fp:
            for i in range(nx+1):
                print(f"{(i-1)*dx:12.4e} {V[i]:12.4e}", file=fp)

        # 時間発展
        for n in range(1, nt+1):

            # 構成関係式
            for i in range(1, nx+1):
                dxV = ( V[i] - V[i-1] ) / dx # 変位速度の空間微分
                S[i] += G[i] * dxV * dt      # 応力の更新

            # 運動方程式
            for i in range(1, nx+1):
                dxS = ( S[i+1] - S[i] ) / dx # 応力の空間微分
                V[i] += dxS / rho[i] * dt    # 変位速度の更新

            # ( n % m == 0 ) は m 回に1回 True になることを利用して出力を間引き
            if n % skip == 0:
                with open(f'{tmpd}/out_{n*dt:06.2f}_.dat', 'w') as fp:
                    for i in range(nx+1):
                        print(f"{(i-1)*dx:12.4e} {V[i]:12.4e}", file=fp)

        # 可視化
        gif = plot_anim(tmpd)

    return gif

関数 `fdm1d` では，前半部分で差分法による数値シミュレーション，後半でその結果の可視化を行います．その中身はほとんど前節と同じですが，入力の**オプション引数** によっていくつかのパラメタを変更できるようにしています．また，何度も実行するので，出力ファイルは都度一時ディレクトリを作って保存し，動画出力後は削除するようにしました．

pythonの関数のオプション変数は，引数として `変数名=値` という形で指定します．これによって，関数の呼び出し時に引数を省略することができますし，引数指定の順序も自由です．`fdm1d` では全ての引数がオプション引数ですので，単に `fdm1d()` とだけ実行すると，前節のパラメタでの数値シミュレーションを実行します．確かめてみましょう．

In [ ]:
fdm1d()

それでは，ここからパラメタを変えた**数値実験**を行います．まずは地震波速度です．`fdm1d()`では，地震波速度 `beta` をリストとして与えると，その速度分布に従って波動伝播をシミュレーションします．

In [ ]:
beta = np.ones(1002)*4.0 # 全体を4.0 km/sで一旦初期化
beta[:300] = 3.5 # 左側30%の領域を 3.5 km/s に
beta[700:] = 2.5 # 右側30%の領域を 2.5 km/s に
fdm1d(beta=beta, nt=501) # 地震波速度が遅くなったぶん時間ステップを少し増やした

上記の例では，初期条件を設定した中央付近はこれまで通りですが，左右1/3ずつの領域は地震波速度を中央部より遅くしています．中央部から左右に広がった地震波が，速度境界で透過波と反射波に分かれています．地震波が速度境界に入射すると，一般にはその境界面における変位とトラクションが連続になる条件を満たすように透過（2次元以上の場合は屈折による方向の変化も）と反射が起こります．その現象はこの数値シミュレーションに自然に含まれている，というわけです．さらに，反射波の振幅が左右で異なること，透過後の波の波長と振幅も左右で異なることがわかります．これらは，波の速度は周波数と波長の積に等しい，という波動方程式の性質から導かれる**分散関係式**によって説明される現象です．

ここまで説明した内容は波動現象の物理として説明できる，いわば「正しい」振る舞いです．しかし，数値シミュレーションはあくまでも『差分』という近似を行っており，場合によっては本来の波動現象とは異なる振る舞いが観測されることもあります．実は上記の数値シミュレーションでも，もともとは左右対称なベル型の波だったのに，特に右側に透過した波ではその形が崩れ，小振幅で振動する尾を引いていることが確認できます．これは「正しくない」振る舞いのひとつです．

このことを強調したパラメタの例が以下の2つの例です．

In [ ]:
fdm1d(dx=1.0, beta=np.ones(202)*4)

In [ ]:
fdm1d(L=2.0, dx=0.4)

どちらの場合も，空間グリッド間隔 `dx` を大きくしました．2つ目の例ではさらに初期条件の幅 `L` も小さくしています．このように，波長にくらべてグリッド間隔が十分に小さくないような場合では，1次元の波動方程式から期待される振る舞いではなく，高周波の振動がより遅れてやってくるような波形が観測されます．これはつまり地震波の速度が周波数に依存してしまっている，ということでもあります．

もうひとつ，こちらは明らかにおかしいと思われる例を紹介しましょう．もともとのパラメタから，地震波速度あるいは時間ステップをほんの僅かにだけ増やしてみます．

In [ ]:
fdm1d(beta=np.ones(1002)*4.001)

In [ ]:
fdm1d(dt=0.05005)

どちらも明らかにおかしいですね．振幅が振動しつつ極端に大きくなり，このまま計算を続けると，あっというまに計算機で再現可能な範囲を超えてしまいます．

差分法による地震波の数値シミュレーションは，かなり自由にパラメタを変えることができ，不均質な媒質中の地震波を再現することができます．しかし，これらの実験を踏まえると，そこには明確な制限がある，ということもわかりました．結論からいうと，

- 波長 $\lambda$ は空間グリッド間隔 $\Delta x$ よりも十分に大きくなければならない（波長条件）
- 時間ステップ $\Delta t$ は空間グリッド間隔と地震波速度の比 $\Delta x / \beta$ に対して十分に小さくなければならない（安定条件）

という2つの条件が課されます．前者の条件が満たされてはじめて，数値シミュレーションで再現される波動現象の地震波速度が現実世界の地震波速度に一致します．逆に，この条件が満たされないと，周波数に依存した地震波速度の望まない変化（**数値分散**）が生じます．一般には，もっとも短い波長が空間グリッドの 5-10 倍あれば安全とされています．

後者の条件は，数値シミュレーションの次元，空間差分公式の次数によって係数が変わりますが，時間ステップの上限が空間グリッド間隔と媒質内の最大の地震波速度で理論的に定まります．その条件をわずかにでも超えると，先の例のような数値不安定が発生し，計算が破綻してしまうのです．

In [ ]:
beta = np.linspace(4, 1., 1002) # 全体を4.0 km/sで一旦初期化
fdm1d(beta=beta, nt=501) # 地震波速度が遅くなったぶん時間ステップを少し増やした